<a href="https://colab.research.google.com/github/wsilveira-splunk/lz78_algorithm/blob/master/lz78.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
%%bash

cd /content/

if [ ! -d "lz78_algorithm" ]; then
  git clone https://github.com/wsilveira-splunk/lz78_algorithm.git
fi

cd lz78_algorithm
git pull origin master
cd -

Updating cc400d3..10698f0
Fast-forward
 files/seq.bin | Bin 0 -> 50000 bytes
 files/seq.mat | Bin 0 -> 8009 bytes
 files/seq.txt |   1 +
 lz78.ipynb    | 163 ++++++++++++++++++++++++++++++++++++++++++++++++++++++++--
 4 files changed, 161 insertions(+), 3 deletions(-)
 create mode 100644 files/seq.bin
 create mode 100644 files/seq.mat
 create mode 100644 files/seq.txt
/content


From https://github.com/wsilveira-splunk/lz78_algorithm
 * branch            master     -> FETCH_HEAD
   cc400d3..10698f0  master     -> origin/master


In [50]:
import os
import numpy as np

In [47]:
def lz78_compress(raw_text):

    dictionary = {'': 0}

    last_seq_letter = ''
    lz78_text = ''

    for letter in raw_text:
        cur_seq_letter = last_seq_letter + letter
        if cur_seq_letter in dictionary:
            last_seq_letter = cur_seq_letter
        else:
            lz78_text += f'{dictionary[last_seq_letter]:012b}' + letter
            dictionary[cur_seq_letter] = len(dictionary)
            last_seq_letter = ''

    if last_seq_letter != '':
        lz78_text += f'{dictionary[last_seq_letter]:012b}'

    #print(dictionary)

    return lz78_text

In [48]:
def lz78_decompress(lz78_text):

    dictionary = {f'{0:012b}': ''}

    raw_text = ''
    seq_letter = ''

    for idx, letter in enumerate(lz78_text, 1):
        if (idx % 13) == 0:
            raw_text += dictionary[seq_letter] + letter
            dictionary[
                f'{len(dictionary):012b}'] = dictionary[seq_letter] + letter
            seq_letter = ''
        else:
            seq_letter += letter

    if seq_letter != '':
        raw_text += dictionary[seq_letter]

    #print(dictionary)

    return raw_text

In [49]:
seq_bin_path = '/content/lz78_algorithm/files/seq.txt'

with open(seq_bin_path) as file:
    data = file.read()
    print('Length of read data: {} bits'.format(len(data)))

    compressed = lz78_compress(data)
    print('Length of compressed data: {} bits'.format(len(compressed)))
    decompressed = lz78_decompress(compressed)
    print('Length of decompressed data: {} bits'.format(len(decompressed)))

    if data == decompressed:
        print('\nDecompressed data matches compressed data - bingo!\n')

Length of read data: 400000 bits
Length of compressed data: 49282 bits
Length of decompressed data: 400000 bits

Decompressed data matches compressed data - bingo!



In [58]:
seq_bin_path = '/content/lz78_algorithm/files/seq.bin'

sum_ones = 0
sum_zeros = 0

with open(seq_bin_path, 'rb') as file:
    data = file.read()
    for _byte in data:
        for _bit in str(bin(_byte)):
            #print(_bit)
            if _bit == '1':
                sum_ones += 1
                
sum_zeros = len(data) * 8 - sum_ones

print('Sum of ones of original data: {}'.format(sum_ones))
print('Sum of zeros of original data: {}'.format(sum_zeros))

Sum of ones of original data: 384812
Sum of zeros of original data: 15188


In [ ]:
def autocorr(x):
    return np.correlate(x, x, mode='full')

def normalize_data(data):
    